In [1]:
#May 28 - for ease for getting data out
import numpy as np
import matplotlib.pyplot as pl
import os

In [60]:
def getData(text):
    #get center z
    data = np.loadtxt(text, skiprows=0, max_rows=2, dtype=('<U16', '<U16'), comments="@")
    data = np.transpose(data)
    z = (float(data[1][1]) - float(data[1][0]))/2 + float(data[1][0])
    zstr = str(round(z, 5))

    #check if errors are linear
    data = np.loadtxt(text, skiprows=2, max_rows=1, dtype=('<U16', '<U16'), comments="@")
    data = np.transpose(data)

    linear = False
    if data[1] == 'linear':
        linear = True
        
    #get data
    data = np.loadtxt(text, skiprows=0)
    data = np.transpose(data)

    masses = data[0]
    phi = data[1]
    err_up = data[2]
    err_down = data[3]

    #putting the errors together in pairs
    errs = []

    if linear:

        for i in range(len(err_up)):
            errs.append((np.log10(err_up[i]), np.log10(err_down[i])))
            phi[i] = np.log10(data[1][i]) #need to convert phi values too

    else:    
        for i in range(len(err_up)):
            errs.append((err_up[i], err_down[i]))

    #check if the masses are evenly spaced
    even = True
    if len(masses) > 2:
        i = 1
        massDif = (masses[1] - masses[0])
        while i < (len(masses)-1) and even:
            if massDif != round((masses[i+1] - masses[i]), 14):
                even = False       
    else:
        even = False
    #get the masses in the correct format
    if not even: #if they are not even, explicitly in a list in 10** form
        masses = np.array([10**m for m in masses])
        massesStr = "["
        for m in masses:
            massesStr = massesStr + "{:.7E}, ".format(m)
        
        massesStr = massesStr.rsplit(',', 1)[0] + "]"
        
        formatThese = [massesStr, str(phi.tolist()), str(errs)]
    else:
        formatThese = [str(masses.tolist()), str(phi.tolist()), str(errs)]
                
    #space each data type nicely
    Strings = ["", "", ""]
    for i, value in enumerate(formatThese):
        
#         print(value)
#         print(len(value))

        #loop through the line while we cut it as long as it's long enough
        while len(value) > 121 : 
            split = 120
            ending = False
            
            if value[split] == ',':
                noComma = False
            else:
                noComma = True
                
            #increase split point until a comma is found
            while noComma & (not ending):
                split +=1
#                 print(split, value[split])
                if value[split] == ',': #found a comma, proceed to splitting
                    noComma = False
                if len(value[split:].split(',')) == 1 or len(value) < split + 1:
                    ending = True

            if not ending:
                split +=1 #go to the next space
                #make a new line
                Strings[i] = Strings[i] + value[:split] + "\n          "
                value = value[split:]
#                 print(len(value))
            else: #got to end of line
                Strings[i] = Strings[i] + value
                value = "" #to end the while loop as value will have lenght 0
        else: #line is too short to split
            Strings[i] = Strings[i] + value
    
    #writting the data to a string
    if even:
        incr = masses[1] - masses[0]
        minM = masses[0]
        maxM = masses[-1] + incr
        
        pyFile = " {}: {{'M': list(10**np.arange({}, {:.3f}, {:.3f})),\n    'phi': {},\n    'err': {}\n   }},".format(zstr, \
                                                    minM, maxM-incr, incr, Strings[1], Strings[2])
#         print(len(np.arange(minM, maxM-incr, incr)))

    else:
        pyFile = " {}: {{'M': {},\n    'phi': {},\n    'err': {}\n   }},".format(zstr,\
                                                    Strings[0], Strings[1], Strings[2])
#         print(len(masses))

    
    return pyFile, z

In [61]:
directory = "../behroozi-2013-data-compilation/smf_ms"
names = ["a"]
zArr = []
for filename in os.listdir(directory):
    if filename.endswith(".smf"): 
        
        name = os.path.join(directory, filename)
        author = filename.split('_')[0]
        if author != names[-1]:
            print(zArr)
            print(f"\n{author}\n")
            zArr = []
            
        names.append(author)
        data, z = getData(name)
        print(data)   
        zArr.append(round(z, 5))
        
    else:
        continue

[]

bouwens

 6.95: {'M': [3.0865532E+09, 1.5020133E+09, 7.3092660E+08, 3.5569173E+08, 1.7309071E+08, 8.4231351E+07, 4.0989608E+07],
    'phi': [-3.62838893005031, -3.75332766665861, -3.18905623622005, -2.77560406136976, -2.54695160369846, -2.41090498583641, -2.13527956994652],
    'err': [(0.2, 0.2), (0.221848749616356, 0.221848749616356), (0.2, 0.2), (0.2, 0.2), (0.2, 0.2), (0.2, 0.2), (0.2, 0.2)]
   },
 7.95: {'M': [1.6471547E+10, 6.9402386E+09, 3.9006268E+09, 1.6435178E+09, 6.9249143E+08, 2.9177925E+08, 1.2294034E+08],
    'phi': [-5.33251957837205, -4.43252385660379, -3.79111622754801, -3.53147891704225, -3.26430718863924, -2.92402389382759, -2.54025284134976],
    'err': [(0.700008537606628, 0.700008537606628), (0.2, 0.2), (0.213879819945081, 0.213879819945081), (0.2, 0.2), (0.2, 0.2), (0.2,
           0.2), (0.2, 0.2)]
   },
[6.95, 7.95]

kslee

 3.96: {'M': [7.9420861E+09, 1.3487597E+10, 2.2905226E+10, 3.8898655E+10, 6.6059394E+10, 1.1218495E+11],
    'phi': [-2.95210787246327,

In [59]:
file = "../behroozi-2013-data-compilation/smf_ms/marchesini_z1.7.smf"

data, z = getData(file)
# print(data)